In [90]:
import os
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [100]:
merged = pd.read_csv("merged_with_gender.tsv", delimiter="\t")
# filter out examples where we failed to infer gender
genders = ['male', 'female']
df_filtered = merged[merged['gender'].isin(genders)]
df_filtered.shape

(53401, 11)

Stolen from stanford people: https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt

In [101]:
stop_words = [w.strip() for w in open("stopwords.txt").readlines()]
stop_words[:10]

['!!', '?!', '??', '!?', '`', '``', "''", '-lrb-', '-rrb-', '-lsb-']

Let's remove gender pronouns since these aren't terribly interesting as predictors

In [118]:
gender_pronouns = ["she", "her", "hers", "his", "he", "guy", "she's", "he's", "commentshe", "commentsshe", "man", "woman"]
stop_words.extend(gender_pronouns)

Merge reviews for individual doctors (we have multiple reviews for each!)

In [105]:
unique_drs = df_filtered["dr-id"].unique()
unique_drs.shape
grouped = df_filtered.groupby('dr-id')
texts, sexes = [], []
for i, dr in grouped:
    cur_text = " ".join(dr['review-text'].values)
    texts.append(cur_text)
    cur_sex = dr["gender"].values[0]
    sexes.append(cur_sex)

In [110]:
len(texts)

16488

How many females v males?

In [109]:
sexes.count("male")/float(len(sexes))

0.7172489082969432

So about 72% male

In [148]:
vectorizer = CountVectorizer(max_features=20000, min_df=25, ngram_range=(1,1), binary=True, stop_words=stop_words)
X = vectorizer.fit_transform(texts)
y = sexes

Note; should probably pull out and pool reviews for physicians -- here we treat individual reviews for the same physician as multiple / independent instances

In [149]:
param_grid = {"C":[.01, .1, 1]}
clf = GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring="f1_macro")

In [150]:
clf.fit(X,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1_macro', verbose=0)

In [151]:
clf.best_score_

0.54194365591947247

In [141]:
def show_most_informative_features(vectorizer, clf, n=50):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [152]:
show_most_informative_features(vectorizer, clf.best_estimator_, n=100)

	-1.7046	seemingly      		2.0930	vague          
	-1.6818	inpatient      		2.0733	abruptly       
	-1.5260	yeast          		1.8747	cocky          
	-1.4709	reference      		1.7473	jerk           
	-1.4381	mary           		1.6032	pursue         
	-1.4349	sweetest       		1.5596	released       
	-1.4067	partum         		1.5320	prostate       
	-1.3976	eyelid         		1.5188	bleed          
	-1.3944	gynecological  		1.5130	covering       
	-1.3731	gown           		1.4894	sensitivity    
	-1.3725	ultra          		1.4754	introduce      
	-1.3722	judgmental     		1.4023	endoscopy      
	-1.3138	sooo           		1.3919	numb           
	-1.3039	assess         		1.3835	relative       
	-1.2656	communicative  		1.3684	delaware       
	-1.2651	sounded        		1.3524	load           
	-1.2547	suicide        		1.3518	playing        
	-1.2503	connected      		1.3170	needles        
	-1.2482	linda          		1.2973	miller         
	-1.2461	1500           		1.2698	canceled       
	-1.2455	argued     